## Import libraries

In [1]:
import pandas as pd
import re
import os
import time
import sys
from datetime import datetime, date
import warnings
import docx
from docx import Document
import urllib.request
import sqlalchemy as sa
import time
warnings.filterwarnings("ignore")

In [4]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
chrome_options = Options()

In [ ]:
basedir = os.getcwd()

os.mkdir(basedir + '/tmpFolder/')

os.chdir(basedir + '/tmpFolder/')

In [ ]:
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")
chrome_options.add_argument('--dns-prefetch-disable')
chrome_options.add_argument('--remote-debugging-port=9222')
#chrome_options.add_experimental_option('prefs', prefs)


## Import files from Browser

In [ ]:
driver = webdriver.Chrome(executable_path=r"\Users\baiba\OneDrive\chromedriver", chrome_options=chrome_options)

driver.maximize_window()
driver.get('https://stat.gov.kz/official/industry/26/statistic/6')

print(driver.title)

driver.find_element_by_xpath("//button[text()='РУС']").click()

driver.find_element_by_xpath("//button[text()='РУС']").click()

print(driver.title)

time.sleep(5)

In [ ]:
try:
    year_sur = datetime.now().year
    driver.find_element_by_xpath(f"//select[@class='field-period']/option[@value={year_sur}]").click()
    #driver.find_element_by_xpath("//select[@class='field-title']/option[@value='1548']").click()
    driver.find_element_by_xpath("//option[text()='Об инфляции в Республике Казахстан']").click()
    print(year_sur)

except NoSuchElementException:
    year_sur = datetime.now().year-1
    driver.find_element_by_xpath(f"//select[@class='field-period']/option[@value={year_sur}]").click()
    #driver.find_element_by_xpath("//select[@class='field-title']/option[@value='1548']").click()
    driver.find_element_by_xpath("//option[text()='Об инфляции в Республике Казахстан']").click()
    print(year_sur)
 

time.sleep(5)

In [ ]:
months = {'январь': '01', 'февраль':'02', 'март':'03', 'апрель':'04', 'май':'05', 'июнь':'06', 
          'июль':'07','август':'08', 'сентябрь':'09', 'октябрь':'10', 'ноябрь':'11', 'декабрь':'12', 
          'январе': '01', 'феврале':'02', 'марте':'03', 'апреле':'04', 'мае':'05', 'июне':'06', 
          'июле':'07','августе':'08', 'сентябре':'09', 'октябре':'10', 'ноябре':'11', 'декабре':'12',
          'января': '01', 'февраля':'02', 'марта':'03', 'апреля':'04', 'мая':'05', 'июня':'06',
          'июля':'07','августа':'08', 'сентября':'09', 'октября':'10', 'ноября':'11', 'декабря':'12',}

In [ ]:
def get_table_by_year(ind_fst, ind_snd):
    df_snd_res = []
    df_fst_res = []
    for i in driver.find_elements_by_xpath('.//div[@class="trud-archive-list"]/a'):
        url = i.get_attribute('href')
        print('The url is: ' + url)
        i.click()
        time.sleep(5)
        i.click()
        print(os.listdir())
        if year_sur != 2014:
            date = pd.to_datetime(months[i.text.split()[1]] + '.01.' + i.text.split()[2])
            #filename = i.text.split()[1] + '_' + i.text.split()[2] + '.docx'
        elif year_sur == 2014:
            date = pd.to_datetime(months[i.text.split()[0].lower()] + '.01.' + i.text.split()[1])
            #filename = i.text.split()[1] + '_' + i.text.split()[2] + '.docx'
            
        print(url, date)
        # w = wc.Dispatch('Word.Application')
        filename = urllib.request.urlopen(url).info().get_filename()
        filename = 'Э'+filename[1:8]+'М'+filename[9:20]+'рус'+filename[23:]

        document = Document(os.path.abspath(filename))

        df_snd_res += get_snd_df(document, ind_snd, date)
        df_fst_res += get_fst_df(document, ind_fst)

        #os.remove(filename)
        
    return df_fst_res, df_snd_res

## Dataframe

In [ ]:
def get_fst_df(document, ind):
    df = pd.DataFrame()
    df['tovar'] = 0
    df['goods'] = 0
    df['food'] = 0
    df['non_food'] = 0
    df['paid_serv'] = 0
    
    
    tables = document.tables
    table = tables[1]
    
    for row in table.rows:
        df.loc[ind] = [cell.text for cell in row.cells][:5]
        ind += 1
    df = check_tovar(df)
    return df.to_dict('records')

In [ ]:
def check_tovar(df):
    m = None
    for i in df.index:
        try:
            m = int(df.loc[i, 'tovar'])
            year = str(m)
        except:
            if df.loc[i, 'tovar'].lower() in months.keys():
                m = pd.to_datetime(months[df.loc[i, 'tovar'].lower()] + '.01.' + year)
            else:
                m = ""
        df.loc[i, 'tovar'] = m
    
    for i in df.index:
        if isinstance(df.loc[i, 'tovar'], datetime):
            continue
        else:
            df = df.drop(index=i)
        
    df.columns = ['date', 'goods', 'food', 'non_food', 'paid_serv'] 
    
    return df

In [ ]:
def get_snd_df(document, ind, date):
    df = pd.DataFrame()
    df['goods'] = 0 
    df['prev_m'] = 0
    df['start_y'] = 0
    df['per_year'] = 0
    
    
    tables = document.tables
    table_4= tables[4]
    table_5 = table_5_df(tables[5])
    

    for row in table_4.rows:
        df.loc[ind] = [cell.text for cell in row.cells][:4]
        ind += 1
    df = df.append(table_5)
    
    if date < pd.to_datetime('2020-09-01'):
        table_6 = table_5_df(tables[6])
        df = df.append(table_6)
        
    df['date'] = date
    return df.to_dict('records')
# import glob

In [ ]:
def table_5_df(table):
    res = []
    row_list = []
    for row in range(0, len(table.rows)):
        for col in iter_visual_cells(table.row_cells(row)):
    #         print(col.text)
            if col.text != '':
                if col.text[0].isalpha():
                    res.append(row_list)
 #                 print(col.text)
                 
                    row_list = []
                    row_list.append(col.text)
                if col.text[0].isdigit():
 #                 print(col.text)
                    row_list.append(col.text)
#                  row_list.append()

    res.append(row_list)
    res = pd.DataFrame(res).loc[:, :3].dropna()
    res.columns = ['goods', 'prev_m', 'start_y', 'per_year']
    
    return res


In [ ]:
def iter_visual_cells(row):
    prior_tc = None
    for cell in row:
        this_tc = cell._tc
        if this_tc is prior_tc:  # skip cells pointing to same <w:tc> element
            continue
        yield cell
        prior_tc = this_tc
        
result_snd = []
result_fst = []

first, second = get_table_by_year( len(result_fst), len(result_snd))
result_fst = first
result_snd = second

In [ ]:
result_df = pd.DataFrame(result_snd).copy()

print(result_df.info())


for i in result_df.index:
    if result_df.loc[i, 'goods'] == '':
        result_df = result_df.drop(index=i)

In [ ]:
result_df.goods = result_df.goods.apply(lambda x: x[0].upper() + x[1:].lower())
for i in ['prev_m', 'start_y', 'per_year']:
    result_df[i] = result_df[i].apply(lambda x: x.replace(',', '.'))
#     result_df[i] = result_df[i].apply(lambda x: 'None' if x == '' else )
result_df.goods = result_df.goods.apply(lambda x:x.replace('\n', ''))

In [ ]:
result_df = result_df.drop(index = result_df[result_df.goods == '    в процентах'].index)
result_df = result_df.drop(index = result_df[result_df.goods == '\xa0'].index)

In [ ]:
def check_goods(x):
    if x == 'Газ сжиженный':
        return 'Газ сжиженный (в баллонах)'
    if x == 'Газ, транспортируемый по распределительным сетям':
        return 'Газ транспортируемый по распределительным сетям'
    if x == 'Мясо':
        return 'Мясо (включая мясопродукты)'
    if x == 'Кефир':
        return 'Кефир 2,5% жирности'
    if x == 'Творог' or x == 'Творог ':
        return 'Творог 5-9% жирности'
    if x == 'Яйца':
        return 'Яйца, I категории'
    if x.find('Яйца') != -1:
        return 'Яйца, I категории'
    if x == 'Горячая  вода':
        return 'Горячая вода'
    if x == 'Жилищные услуги, вода, электроэнергия, газ и' or x == 'Жилищные услуги, вода, электроэнергия, газ и другие виды топлива':
        return 'Жилищные услуги, вода, электроэнергия, газ и прочие виды топлива'
    if x == 'Колбасные изделия':
        return 'Колбасы'
    if x=='Мука пшеничная':
        return 'Мука пшеничная из высшего сорта'
    else:
        return x

In [ ]:
result_df.goods = result_df.goods.apply(lambda x: check_goods(x))


result_fst = pd.DataFrame(result_fst)

result_fst['date'] = result_fst['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

for i in result_fst.columns:
    if i != 'date':
        result_fst[i] = result_fst[i].apply(lambda x: float(x.replace(',', '.')))

In [ ]:
result_df.insert(1, 'date_rec', result_df['date'])

result_df.drop('date', axis=1, inplace=True)

result_df.insert(1, 'category', '')

result_df['category']=result_df['goods'].map(df_dict)

result_df.columns=['title', 'category', 'date_rec', 'index_to_prev_month', 'index_to_jan_cur_year', 'index_to_dec_prev_year']
result_df['date_rec'] = result_df['date_rec'].apply(lambda x: x.date())

In [ ]:
print('This is the titles')
print(result_df[result_df['category'].isna()]['title'].unique())

print(result_df.info())
print(result_fst.info())

In [ ]:
table_name_1 = 'price_index_by_date_to_last_month'

table_name_2 = 'price_index_by_goods_services'

## Move dataframe to sql

In [ ]:
engine = sa.create_engine(
            "postgresql+psycopg2://" + DB_USERNAME + ":" + DB_PASSWORD + "@" + DB_HOST + ":5432/" + DB_DATABASE + "",
            echo=False, connect_args={'options': "-c search_path=anl"})
conn = engine.connect()

df_orig_1 = pd.read_sql_table(table_name_1, conn)

df_upl_1 = df_orig_1.append(result_fst, ignore_index=True)

df_upl_1.drop_duplicates(inplace=True, ignore_index=True)

df_upl_1.to_sql(table_name_1, conn, if_exists='replace', index=False, method='multi')

df_orig_2 = pd.read_sql_table(table_name_2, conn)

df_orig_2['date_rec']=df_orig_2['date_rec'].apply(lambda x: x.date())

df_upl_2 = df_orig_2.append(result_df, ignore_index=True)

df_upl_2.drop_duplicates(inplace=True, ignore_index=True)

df_upl_2.to_sql(table_name_2, conn, if_exists='replace', index=False, method='multi')

print('Mission accomplished!')